# DSSP Demo

This demo shows how to create and query a dssp dataset.

DSSP is a database of secondary structure assigmnets for all protein entries in the Protein Data Bank (PDB).

[DSSP Website](http://swift.cmbi.ru.nl/gv/dssp/)

## Imports

In [1]:
from pyspark import SQLContext
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import secondaryStructureExtractor
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.mappers import StructureToPolymerChains
import time

#### Configure Spark 

In [2]:
spark = SparkSession.builder.appName("DSSPDemo").getOrCreate()

2022-01-23 14:19:30 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Download single protein (1STP)

In [3]:
pdbIds = ["1STP"]

pdb = mmtfReader.download_mmtf_files(pdbIds).cache()

## Flatmap to polymer chains

In [4]:
pdb = pdb.flatMap(StructureToPolymerChains())

## Extract Secondary Structures

In [5]:
ds = secondaryStructureExtractor.get_dataset(pdb)

ds.printSchema()
ds.show(2, False)

root
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- alpha: float (nullable = false)
 |-- beta: float (nullable = false)
 |-- coil: float (nullable = false)
 |-- dsspQ8Code: string (nullable = false)
 |-- dsspQ3Code: string (nullable = false)



[Stage 4:============================================>              (3 + 1) / 4]

+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|structureChainId|sequence                                                                                                                                                       |alpha     |beta    |coil      |dsspQ8Code                                                                                                                                                     |dsspQ3Code                                                                                           

## Terminate Spark

In [6]:
spark.stop()